<a href="https://colab.research.google.com/github/PythonDecorator/Amazon-EC2-with-github-actions/blob/main/Loan_Default_and_Credit_Risk_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Statement

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# for displaying markdown
from IPython.display import display, Markdown

# sklearn
from sklearn.model_selection import train_test_split

# Mount GDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [22]:
file_path_accepted = '/content/drive/MyDrive/MY WORK/ML/LoanDefault/accepted_2007_to_2018Q4.csv'
file_path_rejected = '/content/drive/MyDrive/MY WORK/ML/LoanDefault/rejected_2007_to_2018Q4.csv'

In [4]:
class DataLoader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.df = None

    def load_csv(self, usecols: list = [], dtypes: dict = {}, chunksize: int = 0, nrows: int = 0):
        """
        Load CSV with optional column selection, dtypes, chunking, and limited rows.
        """
        if nrows:
            self.df = pd.read_csv(self.filepath, usecols=usecols or None, dtype=dtypes or None, nrows=nrows)
        elif chunksize:
            chunks = []
            for chunk in pd.read_csv(self.filepath, usecols=usecols or None, dtype=dtypes or None, chunksize=chunksize):
                chunks.append(chunk)
            self.df = pd.concat(chunks, ignore_index=True)
        else:
            self.df = pd.read_csv(self.filepath, usecols=usecols or None, dtype=dtypes or None)
        return self.df

    def preview_columns(self, start_col=0, ncols=10, nrows=10):
        """
        Preview a subset of the dataframe:
        - First nrows
        - Columns from start_col to start_col + ncols
        """
        if self.df is None:
            raise ValueError("Dataframe not loaded. Call load_csv() first.")
        end_col = start_col + ncols
        return self.df.iloc[:nrows, start_col:end_col]


In [5]:
cols = [
    # 'id',
    'loan_amnt',
    'funded_amnt',
    'funded_amnt_inv',
    'term',
    'int_rate',
    'installment',
    'grade',
    'sub_grade',
    # 'emp_title',
    'emp_length',
    'home_ownership',
    'annual_inc',
    'verification_status',
    # 'issue_d',
    'loan_status',
    'pymnt_plan',
    # 'url',
    'purpose',
    # 'title',
    # 'zip_code',
    # 'addr_state',
    'dti',
    'delinq_2yrs',
    # 'earliest_cr_line',
    'fico_range_low',
    'fico_range_high',
    'inq_last_6mths',
    'open_acc',
    'pub_rec',
    'revol_bal',
    'revol_util',
    'total_acc',
    'initial_list_status',
    'out_prncp',
    'out_prncp_inv',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_prncp',
    'total_rec_int',
    'total_rec_late_fee',
    'recoveries',
    'collection_recovery_fee',
    # 'last_pymnt_d',
    'last_pymnt_amnt',
    # 'last_credit_pull_d',
    'last_fico_range_high',
    'last_fico_range_low',
    'collections_12_mths_ex_med',
    'mths_since_last_delinq',
    'policy_code',
    'application_type',
    'acc_now_delinq',
    'tot_coll_amt',
    'tot_cur_bal',
    'open_acc_6m',
    'open_act_il',
    'open_il_12m',
    'open_il_24m',
    'mths_since_rcnt_il',
    'total_bal_il',
    # 'il_util',
    'open_rv_12m',
    'open_rv_24m',
    'max_bal_bc',
    'all_util',
    'total_rev_hi_lim',
    'inq_fi',
    'total_cu_tl',
    'inq_last_12m',
    'acc_open_past_24mths',
    'avg_cur_bal',
    'bc_open_to_buy',
    'bc_util',
    'chargeoff_within_12_mths',
    'delinq_amnt',
    'mo_sin_old_il_acct',
    'mo_sin_old_rev_tl_op',
    'mo_sin_rcnt_rev_tl_op',
    'mo_sin_rcnt_tl',
    'mort_acc',
    'mths_since_recent_bc',
    'mths_since_recent_inq',
    'num_accts_ever_120_pd',
    'num_actv_bc_tl',
    'num_actv_rev_tl',
    'num_bc_sats',
    'num_bc_tl',
    'num_il_tl',
    'num_op_rev_tl',
    'num_rev_accts',
    'num_rev_tl_bal_gt_0',
    'num_sats',
    # 'num_tl_120dpd_2m',
    'num_tl_30dpd',
    'num_tl_90g_dpd_24m',
    'num_tl_op_past_12m',
    'pct_tl_nvr_dlq',
    'percent_bc_gt_75',
    'pub_rec_bankruptcies',
    'tax_liens',
    'tot_hi_cred_lim',
    'total_bal_ex_mort',
    'total_bc_limit',
    'total_il_high_credit_limit',
    'hardship_flag',
    'disbursement_method',
    'debt_settlement_flag'
]

In [6]:
dtypes = {
    'loan_amnt': 'float64',
    'funded_amnt': 'float64',
    'funded_amnt_inv': 'float64',
    'term': 'object',
    'int_rate': 'float64',
    'installment': 'float64',
    'grade': 'object',
    'sub_grade': 'object',
    'emp_title': 'object',
    'emp_length': 'object',
    'home_ownership': 'object',
    'annual_inc': 'float64',
    'verification_status': 'object',
    'issue_d': 'object',
    'loan_status': 'object',
    'pymnt_plan': 'object',
    'purpose': 'object',
    'dti': 'float64',
    'delinq_2yrs': 'float64',
    'earliest_cr_line': 'object',
    'fico_range_low': 'float64',
    'fico_range_high': 'float64',
    'inq_last_6mths': 'float64',
    'open_acc': 'float64',
    'pub_rec': 'float64',
    'revol_bal': 'float64',
    'revol_util': 'float64',
    'total_acc': 'float64',
    'initial_list_status': 'object',
    'out_prncp': 'float64',
    'out_prncp_inv': 'float64',
    'total_pymnt': 'float64',
    'total_pymnt_inv': 'float64',
    'total_rec_prncp': 'float64',
    'total_rec_int': 'float64',
    'total_rec_late_fee': 'float64',
    'recoveries': 'float64',
    'collection_recovery_fee': 'float64',
    'last_pymnt_d': 'object',
    'last_pymnt_amnt': 'float64',
    'last_credit_pull_d': 'object',
    'last_fico_range_high': 'float64',
    'last_fico_range_low': 'float64',
    'collections_12_mths_ex_med': 'float64',
    'mths_since_last_delinq': 'float64',
    'policy_code': 'float64',
    'application_type': 'object',
    'acc_now_delinq': 'float64',
    'tot_coll_amt': 'float64',
    'tot_cur_bal': 'float64',
    'open_acc_6m': 'float64',
    'open_act_il': 'float64',
    'open_il_12m': 'float64',
    'open_il_24m': 'float64',
    'mths_since_rcnt_il': 'float64',
    'total_bal_il': 'float64',
    'open_rv_12m': 'float64',
    'open_rv_24m': 'float64',
    'max_bal_bc': 'float64',
    'all_util': 'float64',
    'total_rev_hi_lim': 'float64',
    'inq_fi': 'float64',
    'total_cu_tl': 'float64',
    'inq_last_12m': 'float64',
    'acc_open_past_24mths': 'float64',
    'avg_cur_bal': 'float64',
    'bc_open_to_buy': 'float64',
    'bc_util': 'float64',
    'chargeoff_within_12_mths': 'float64',
    'delinq_amnt': 'float64',
    'mo_sin_old_il_acct': 'float64',
    'mo_sin_old_rev_tl_op': 'float64',
    'mo_sin_rcnt_rev_tl_op': 'float64',
    'mo_sin_rcnt_tl': 'float64',
    'mort_acc': 'float64',
    'mths_since_recent_bc': 'float64',
    'mths_since_recent_inq': 'float64',
    'num_accts_ever_120_pd': 'float64',
    'num_actv_bc_tl': 'float64',
    'num_actv_rev_tl': 'float64',
    'num_bc_sats': 'float64',
    'num_bc_tl': 'float64',
    'num_il_tl': 'float64',
    'num_op_rev_tl': 'float64',
    'num_rev_accts': 'float64',
    'num_rev_tl_bal_gt_0': 'float64',
    'num_sats': 'float64',
    'num_tl_30dpd': 'float64',
    'num_tl_90g_dpd_24m': 'float64',
    'num_tl_op_past_12m': 'float64',
    'pct_tl_nvr_dlq': 'float64',
    'percent_bc_gt_75': 'float64',
    'pub_rec_bankruptcies': 'float64',
    'tax_liens': 'float64',
    'tot_hi_cred_lim': 'float64',
    'total_bal_ex_mort': 'float64',
    'total_bc_limit': 'float64',
    'total_il_high_credit_limit': 'float64',
    'hardship_flag': 'object',
    'disbursement_method': 'object',
    'debt_settlement_flag': 'object'
}

In [7]:
data_loader = DataLoader(file_path_accepted)
accepted_df = data_loader.load_csv(usecols=cols, dtypes=dtypes, chunksize=100_000)

In [8]:
data_loader.preview_columns()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,10+ years,MORTGAGE
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE
5,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,4 years,RENT
6,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,10+ years,MORTGAGE
7,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,10+ years,MORTGAGE
8,10000.0,10000.0,10000.0,36 months,6.49,306.45,A,A2,6 years,RENT
9,8000.0,8000.0,8000.0,36 months,11.48,263.74,B,B5,10+ years,MORTGAGE


# Data Visualization EDA

In [9]:
class DataVisualization:
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def basic_info(self):
        """Show shape, columns, head/tail, duplicates."""
        display(Markdown("## Basic Info"))
        display(Markdown(f"- Shape: **{self.data.shape}**"))

        display(Markdown("### Top 5 rows"))
        display(self.data.head())

        display(Markdown("### Duplicate rows"))
        duplicates = self.data[self.data.duplicated(keep=False)]
        display(duplicates)

    def export_combined_data_summary(self, filename: str = "data_summary.csv", data = None):
        """Summary of each column saved to CSV and displayed as Markdown table."""
        if data is None:
            data = self.data

        summary = pd.DataFrame({
            "Column Name": self.data.columns,
            "First Row": self.data.iloc[0].values,
            "Last Row": self.data.iloc[-1].values,
            "Data Type": self.data.dtypes.values,
            "Missing Count": self.data.isnull().sum().values,
            "Missing %": (self.data.isnull().sum() / len(self.data) * 100).round(2).values,
            "Unique Values": self.data.nunique().values
        })
        summary.to_csv(filename, index=False)
        # display(Markdown(f"✅ Combined data summary saved to **{filename}**"))

        # md_table = "## Combined Data Summary\n"
        # md_table += "| Column Name | First Row | Last Row | Data Type | Missing Count | Missing % | Unique Values |\n"
        # md_table += "|---|---|---|---|---|---|---|\n"
        # for _, row in summary.iterrows():
        #     md_table += f"| {row['Column Name']} | {row['First Row']} | {row['Last Row']} | {row['Data Type']} | {row['Missing Count']} | {row['Missing %']} | {row['Unique Values']} |\n"
        # display(Markdown(md_table))

    def describe_numeric(self, columns=None):
        """Describe numeric columns (rounded to 2 decimals)."""
        cols = columns if columns else self.data.select_dtypes(include=np.number).columns
        display(Markdown("## Numeric Summary"))
        display(self.data[cols].describe().round(2))

    def outliers_zscore(self, column: str, threshold=3):
        """Show rows that are outliers in `column` by Z-score."""
        display(Markdown(f"## Outliers in `{column}` (|Z|>{threshold})"))
        z_scores = np.abs(stats.zscore(self.data[column].dropna()))
        outliers = self.data.loc[z_scores.index[z_scores > threshold]]
        display(outliers)

    def unique_values(self, columns: list):
        """Display unique values for a list of columns as a Markdown table."""
        display(Markdown("## Unique Values"))
        max_len = max(self.data[col].nunique() for col in columns)
        md_table = "| " + " | ".join(columns) + " |\n"
        md_table += "| " + " | ".join(["---"] * len(columns)) + " |\n"
        for i in range(max_len):
            row = []
            for col in columns:
                uniques = self.data[col].unique()
                row.append(str(uniques[i]) if i < len(uniques) else "-")
            md_table += "| " + " | ".join(row) + " |\n"
        display(Markdown(md_table))

    def plot_bar_kde(self, columns: list):
        """Plot histogram + KDE for numeric columns."""
        for col in columns:
            plt.figure(figsize=(6,4))
            sns.histplot(self.data[col], kde=True)
            plt.title(f"Distribution of {col}")
            plt.show()

    def plot_scatter(self, x: str, y: str, hue = None):
        """Plot scatter between two numeric columns, optionally with hue."""
        plt.figure(figsize=(6,4))
        sns.scatterplot(data=self.data, x=x, y=y, hue=hue, alpha=0.6)
        plt.title(f"{x} vs {y}")
        plt.show()

    def run(self, numeric_cols=None, outlier_col=None, unique_cols=None, bar_cols=None, scatter=None):
        """
        Run a quick full EDA pipeline.
        """
        self.basic_info()
        self.export_combined_data_summary()

        if numeric_cols:
            self.describe_numeric(numeric_cols)
        if outlier_col:
            self.outliers_zscore(outlier_col)
        if unique_cols:
            self.unique_values(unique_cols)
        if bar_cols:
            self.plot_bar_kde(bar_cols)
        if scatter:
            self.plot_scatter(*scatter)




In [10]:
eda = DataVisualization(accepted_df)
eda.run()


## Basic Info

- Shape: **(2260701, 96)**

### Top 5 rows

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,...,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,10+ years,MORTGAGE,...,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N,Cash,N
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N


### Duplicate rows

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
749520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
749521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [36]:
class DataCleaner:
    """Class for cleaning and preprocessing a DataFrame."""

    def __init__(self, data: pd.DataFrame):
        # copy the data to avoid changing the original data
        self.data = data.copy()

    def remove_duplicates(self):
        """Remove duplicate rows from the DataFrame."""
        self.data.drop_duplicates(inplace=True)

    def drop_cols_with_over_30_missing(self, threshold: float = 0.3):
        """
        Drop columns with more than `threshold` missing values.
        Default threshold = 0.3 (30%).
        """
        missing_fraction = self.data.isna().mean()
        cols_to_drop = missing_fraction[missing_fraction > threshold].index
        self.data.drop(columns=cols_to_drop, inplace=True)

    def normalize_emp_length_to_numeric(self):
        """
        Convert 'emp_length' column to numeric values.
        '10+ years' -> 10
        '3 years'   -> 3
        -> np.nan (then filled)
        """
        if 'emp_length' in self.data.columns:
            def convert_emp_length(val):
                if pd.isna(val):
                    return np.nan
                val = str(val).lower().strip()
                if '<' in val:          # '< 1 year'
                    return 0.5
                if '10+' in val:        # '10+ years'
                    return 10
                if 'n/a' in val or 'na' == val:
                    return np.nan
                digits = ''.join(c for c in val if c.isdigit())
                return float(digits) if digits else np.nan

            self.data['emp_length'] = self.data['emp_length'].apply(convert_emp_length)

            # fill missing with mean (or other estimate)
            mean_val = self.data['emp_length'].mean()
            self.data['emp_length'].fillna(mean_val, inplace=True)

    def fill_other_missing_values(self):
        """
        Fill missing values:
        - For numeric columns, fill with mean.
        - For object/categorical columns, fill with mode.
        """
        for col in self.data.columns:
            if self.data[col].dtype in [np.float64, np.float32, np.int64, np.int32]:
                self.data[col].fillna(self.data[col].mean(), inplace=True)
            else:
                mode_val = self.data[col].mode()
                if not mode_val.empty:
                    self.data[col].fillna(mode_val[0], inplace=True)

    def convert_to_categorical(self, columns=None):
        """
        Convert specified columns to categorical dtype.
        If columns is None, convert all object/string columns.
        """
        if columns is None:
            # Convert all object dtype columns
            cols_to_convert = self.data.select_dtypes(include='object').columns
        else:
            cols_to_convert = columns

        for col in cols_to_convert:
            self.data[col] = self.data[col].astype('category')

    def export_combined_data_summary(self, filename: str = "final_data_summary.csv"):
        """Summary of each column saved to CSV and displayed as Markdown table."""
        summary = pd.DataFrame({
            "Column Name": self.data.columns,
            "First Row": self.data.iloc[0].values,
            "Last Row": self.data.iloc[-1].values,
            "Data Type": self.data.dtypes.values,
            "Missing Count": self.data.isnull().sum().values,
            "Missing %": (self.data.isnull().sum() / len(self.data) * 100).round(2).values,
            "Unique Values": self.data.nunique().values
        })
        summary.to_csv(filename, index=False)

    def clean(self, filename: str = "final_data_summary.csv"):
        """Run the full cleaning pipeline."""
        self.remove_duplicates()
        self.drop_cols_with_over_30_missing()  # default 30% threshold
        self.normalize_emp_length_to_numeric()
        self.fill_other_missing_values()
        self.convert_to_categorical()
        self.export_combined_data_summary(filename=filename)

    def get_cleaned_data(self):
        """Get the cleaned DataFrame."""
        print(f"✅ Data cleaning complete! - Shape {self.data.shape}")
        return self.data


# Get Accepted Data

In [12]:
# cleaner instance
data_cleaner = DataCleaner(accepted_df)

# perform data cleaning
data_cleaner.clean()

# get the cleaned data
cleaned_accepted_df = data_cleaner.get_cleaned_data()

✅ Data cleaning complete! - Shape (2260669, 82)


In [33]:
# check the target col, loan_status col
cleaned_accepted_df['loan_status'].value_counts()

,count
loan_status,
Fully Paid,1076752
Current,878317
Charged Off,268559
Late (31-120 days),21467
In Grace Period,8436
Late (16-30 days),4349
Does not meet the credit policy. Status:Fully Paid,1988
Does not meet the credit policy. Status:Charged Off,761
Default,40


# Loan Status Distribution

- **Good Loans (Fully Paid + DNM FP):** 1,078,740 (~78%)
- **Bad Loans (Charged Off + Default + Late + In Grace + DNM CO):** 303,612 (~22%)
- **Current Loans:** 878,317 (kept as separate test set)

## Observations

- Bad loans are a minority → class imbalance is expected.
- Model will see more “good” examples than “bad” during training.
- Current loans are kept separate to evaluate future defaults.

## Handling Class Imbalance

- Use **class weights** in the model (`class_weight='balanced'` in sklearn).  
- **Oversample** bad loans (SMOTE, RandomOverSampler).  
- **Undersample** good loans (with caution).  
- Evaluate with metrics robust to imbalance: **ROC-AUC**, **Precision-Recall**.


# Split cleaned data to test and train

In [ ]:
class DefaultDataSplitter:
    """
    A class for preparing and splitting Lending Club loan data into training and test sets.

    - Training set: loans with a final outcome (Fully Paid, Charged Off, Default, etc.)
    - Test set: loans with status 'Current'
    - Creates binary target 'default_flag' (1 = bad loan, 0 = good loan)
    """

    def __init__(self, data: pd.DataFrame):
        # Copy data to avoid modifying original
        self.data = data.copy()

        # Placeholders
        self.train_df = None
        self.current_df = None

    def prepare_data(self):
        """Create default_flag and separate training (finalized) and test (Current) sets."""
        bad_statuses = [
            'Charged Off',
            'Default',
            'Does not meet the credit policy. Status:Charged Off',
            'Late (31-120 days)',
            'Late (16-30 days)',
            'In Grace Period'
        ]
        good_statuses = [
            'Fully Paid',
            'Does not meet the credit policy. Status:Fully Paid'
        ]

        # Training data: finalized loans only
        self.train_df = self.data[self.data['loan_status'].isin(bad_statuses + good_statuses)].copy()
        self.train_df['default_flag'] = np.where(self.train_df['loan_status'].isin(bad_statuses), 1, 0)

        # Test data: current loans
        self.current_df = self.data[self.data['loan_status'] == 'Current'].copy()

        return self.train_df, self.current_df

    def split_train_test(self, test_size: float = 0.2, random_state: int = 42):
        """Split finalized loans into training and validation sets."""
        if self.train_df is None:
            raise ValueError("Call prepare_data() first to create training and test sets.")

        X = self.train_df.drop(['loan_status', 'default_flag'], axis=1)
        y = self.train_df['default_flag']

        x_train, x_val, y_train, y_val = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )

        return x_train, x_val, y_train, y_val

    def get_train_and_current(self):
        """Return the full training DataFrame and the current loans DataFrame."""
        if self.train_df is None or self.current_df is None:
            raise ValueError("Call prepare_data() first.")
        return self.train_df, self.current_df
